In [1]:
!pip install pyunpack
!pip install patool

In [2]:
import os
os.system('apt-get install p7zip')

이 커널은 Keras를 사용하여 CNN 구축을 실험하고자 하는 초심자를 위한 것입니다. 이 커널을 사용하면 좋은 점수를 기대하고 Keras도 배울 수 있습니다. Keras는 모델을 초기화하고 원하는 레이어를 계속 쌓을 수 있는 간단한 프레임워크입니다. 심층 신경망을 매우 쉽게 구훅할 수 있습니다.

In [3]:
from pyunpack import Archive
import shutil
import pandas as pd
import numpy as np

if not os.path.exists('/kaggle/train/'):
    os.makedirs('/kaggle/train/')
    
Archive('/kaggle/input/statoil-iceberg-classifier-challenge/train.json.7z').extractall('/kaggle/train/')

if not os.path.exists('/kaggle/test/') :
    os.makedirs('/kaggle/test/')

Archive('/kaggle/input/statoil-iceberg-classifier-challenge/test.json.7z').extractall('/kaggle/test/')

for dirname, _, filenames in os.walk('/kaggle/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

train = pd.read_json('../train/data/processed/train.json')
test = pd.read_json('../test/data/processed/test.json')

## 데이터 소개
Sentinet-1 sat은 지상 약 680km에 있습니다. 특정 입사각에서 신호 펄스를 보낸 다음 다시 코딩합니다. 기본적으로 이러한 반사 신호를 후방 산란이라고 합니다. 우리에게 주어진 데이터는 다음과 같이 주어진 전통적인 형태의 후방 산란 계수인 후방 산란 계수입니다. :   

$$\sigma_0(dB) = \beta_0(dB) + 10\log10[\frac{\sin(ip)}{\sin(ic)}]$$

1. ip = 특정 픽셀에 대한 입사각
2. ic = 이미지 중심에 대한 입사각
3. K = 상수

우리는 데이터에서 직접 𝜎0를 받았습니다.

### 이제 𝜎0의 기능에 대해 알아보자.   
기본적으로 𝜎0는 신호가 산란되는 표면에 따라 다릅니다. 예를 들어 특정 입사각의 경우 다음과 같이 다양합니다.:
* WATER..........SETTLEMENTS..........AGRICULTURE..........BARREN..........
1. HH:-27.001..........2.70252.........-12.7952..........-17.25790909
2. HH:-28.035..........-20.2665..........-21.4471..........-20.019

보시다시피 HH 구성 요소는 많이 다르지만 HV는 그렇지 않습니다. 
#### 배의 산란에 대한 데이터는 없지만 금속 물체이므로 얼음 물체와 비교하여 달라야 합니다.

### WTF는 HH HV입니까?
좋아, 그래서 이 Sentinal Settalite는 RISTSAT(인도 원격 감지 토)와 동일하며 V 편광이 아닌 H 편광에서만 핑을 전송합니다. 이러한 H-ping이 흩어지고 물체가 편광을 변경하여 H와 V의 혼합으로 반환됩니다. Sentinel에는 H-송신기만 있는 때문에 반환 신호는 HH 및 HV의 형태만 있습니다. VV가 제공되지 않는 이유는 묻지 마십시오. (Sentinel에는 V-ping 송신기가 없기 때문에)

이제 기능에 대해 설명하겠습니다. 이 데모 코드의 목적을 위해 두 밴드를 모두 추출하고 이 밴드의 평균을 세 번째 채널로 사용하여 3채널 RGB를 생성합니다.

In [4]:
# Generate the training data
# Create 3 bands having HH, HV and avg of both
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_1']])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75,75) for band in train['band_2']])
X_train=np.concatenate([X_band_1[:,:,:,np.newaxis], X_band_2[:,:,:,np.newaxis], ((X_band_1+X_band_2)/2)[:,:,:,np.newaxis]], axis=-1)

In [5]:
# Take a look at a iceberg
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
def plotmy3d(c, name):
    
    data=[go.Surface(z=c)]
    layout=go.Layout(
        title=name,
        autosize=False,
        width=700,
        height=700,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90))
    fig=go.Figure(data=data, layout=layout)
    py.iplot(fig)
plotmy3d(X_band_1[12,:,:], 'iceberg')

이것은 우리가 가지고 있는 멋진 빙산입니다. 레이더 데이터에서 빙산의 모양은 여기에 표시된 것처럼 산과 비슷할 것임을 기억하십시오. 이것은 실제 이미지가 아니라 레이더에서 산란되기 때문에 모양에 이러한 피크와 왜곡이 있을 것입니다. 배의 모양은 점과 같을 것이고, 길쭉한 점과 같을 수 있습니다. 여기에서 구조적 차이가 발생하고 CNN을 사용하여 이러한 차이를 활용할 수 있습니다. 레이더의 후방 산란을 사용하여 합성 이미지를 만들 수 있다면 도움이 될 것입니다.

In [6]:
plotmy3d(X_band_1[14,:,:], 'Ship')

이것은 배입니다. 길쭉한 점처럼 보입니다. 우리는 배의 모양을 시각화하기 위한 이미지의 해상도가 많지 않습니다. 그러나 CNN이 도움을 줄 수 있습니다. 다음과 같은 선박 빙산 분류에 대한 논문은 거의 없습니다. 
http://elib.dlr.de/99079/2/2016_BENTES_Frost_Velotto_Tings_EUSAR_FP.pdf
그러나 그들의 데이터는 훨씬 더 나은 해상도를 가지고 있으므로 그들이 사용한 CNN이 여기에 적합할 것이라고 생각하지 않습니다.

Keras를 사용하여 CNN 구축으로 돌아가십시오. 다른 것보다 훨씬 더 나은 프레임워크.

In [7]:
# Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [8]:
# define out model
def getModel():
    # Building the model
    gmodel=Sequential()
    # Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(75,75,3)))
    gmodel.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
    gmodel.add(Dropout(0.2))
    
    # Conv Layer 2
    gmodel.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    gmodel.add(Dropout(0.2))
    
    # Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    gmodel.add(Dropout(0.2))
    
    # Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
    gmodel.add(Dropout(0.2))
    
    # Flatten the data for upcoming dense layers
    gmodel.add(Flatten())
    
    # Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))
    
    # Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))
    
    # Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))
    
    mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode='min')
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = '.model_weights.hdf5'
callbacks = get_callbacks(filepath=file_path, patience=5)

In [10]:
from sklearn.model_selection import train_test_split

target_train=train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=1, train_size=0.75)

In [11]:
# Without denoising, core features.
import os
gmodel=getModel()
gmodel.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=50,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

### 여기 점수가 다를 수 있지만 LB에서 잘 작동합니다. 이 개발자는 0.210점수를 얻었습니다.

In [12]:
gmodel.load_weights(filepath=file_path)
score = gmodel.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [14]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test['band_1']])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75,75) for band in test['band_2']])
X_test=np.concatenate([X_band_test_1[:,:,:,np.newaxis],
                      X_band_test_2[:,:,:,np.newaxis],
                      ((X_band_test_1+X_band_test_2)/2)[:,:,:,np.newaxis]], axis=-1)
predicted_test=gmodel.predict(X_test)

In [15]:
submission=pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)

## 결론
점수를 높이기 위해 Speckle 필터링, indicence angle normalization 및 기타 전처리를 시도했지만 작동하지 않는 것 같습니다.

이 커널을 사용하여 상위 10위 안에 들 수는 없습니다. 그래서 여기에 하나의 정보가 있습니다. 테스트 데이터 세트에는 8,000개의 이미지가 포함되어 있습니다. 이를 악용할 수 있습니다. 예측을 높이기 위해 준레이블링을 수행할 수 있습니다. 다음은 이와 관련된 기사입니다.:https://towardsdatascience.com/simple-explanation-of-semi-supervised-learning-and-pseudo-labeling-c2218e8c769b